In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
import math
import sklearn as sk
from sklearn.linear_model import LogisticRegression as LR

%matplotlib inline

sns.set(style="dark")
plt.style.use("ggplot")

# Dataset information

**1. Uber Pickups in New York City**
- Each record is an uber order for 2014 (important columns include date/time, lat, lon)
    - uber-raw-data-apr14.csv
    - uber-raw-data-aug14.csv
    - uber-raw-data-jul14.csv
    - uber-raw-data-jun14.csv
    - uber-raw-data-may14.csv
    - uber-raw-data-sep14.csv
- Each record is an uber order for 2015 (important columns include date/time, lat, lon)
    - uber-raw-data-janjune-15.csv
- Questions to answer: 
    - How has inflation affected the number of rides in 2014 and 2015?

**2. uber fares**
- Each record is an uber order for years 2009-2015 (important columns include fare amount, pickup date, passenger count)
- Questions to answer: 
    - How has inflation affected the fare amount per passenger?
    - How has inflation affected the number of rides from 2009 to 2015?
        - Much less data per month than (1).
        
**3. sf gas data**
- Each record is the dollar per gallon price of gas for each week from 2000 to 2022
- Combine this data with the above 2.
    
**4. inflation_expectation**
- Contains monthly inflationary expectations
- To be used as the instrument variable for the regression

**5. unemployment_rate**
- Contains monthly unemployment rates
- To be used as a confounding variable

# How does inflation affect the number of trips?

Objective: Use Uber as a proxy to find how the number of orders for gig companies is impacted by inflation.

Regression: $ Y_{orders} = \beta_{0} + \beta_{1}Z_{gas ~ price} + \beta_{2}X_{confounder}$
- Y = number of Uber trips
- Z = gas price
- X = confounders (affects Y and Z)
    - Uber price
    - Unemployment
- W = instrument variable
    - ? (interest rates, inflationary expectations, output gap, pandemic, budget deficit)
    - Instrument: Inflationary expectations
        - Key assumptions: 
            - Gas prices are highly correlated with inflation since gas prices make up a large proportion of energy spending. Therefore, gas prices closely follow actual inflation.
            - Inflationary expectations only affect actual inflation (and gas prices); businesses and individuals don't incorporate inflationary expectations in their decision-making

Below is the general diagram for causality, which will inform our IV regression.

<img src='causal_diagram.png' width=400 >

## Data cleaning

In [2]:
df_2015 = pd.read_csv('Uber Pickups in New York City/uber-raw-data-janjune-15.csv')#.sample(frac=0.2, replace=False)

In [3]:
df_2015['Pickup_date'] = pd.to_datetime(df_2015['Pickup_date'])
df_2015['month'] = df_2015['Pickup_date'].dt.month
df_2015['year'] = df_2015['Pickup_date'].dt.year
df_2015['day'] = df_2015['Pickup_date'].dt.day
df_2015 = df_2015.groupby(['month', 'year', 'day']).agg(len)
df_2015 = df_2015.iloc[:, 0].reset_index().rename({'Dispatching_base_num': 'number of rides'}, axis=1)
df_2015

,month,year,day,number of rides
0,1,2015,1,58045
1,1,2015,2,40287
2,1,2015,3,59844
3,1,2015,4,41943
4,1,2015,5,43678
...,...,...,...,...
176,6,2015,26,99508
177,6,2015,27,136193
178,6,2015,28,93108
179,6,2015,29,74430


In [4]:
df_inflation = pd.read_csv('inflation_expectation.csv')

In [5]:
df_inflation = df_inflation.rename({'MICH': 'expected inflation'}, axis='columns')
df_inflation['month'] = pd.to_datetime(df_inflation['DATE']).dt.month
df_inflation['year'] = pd.to_datetime(df_inflation['DATE']).dt.year
df_inflation['day'] = pd.to_datetime(df_inflation['DATE']).dt.day
df_inflation = df_inflation.groupby(['month', 'year', 'day']).agg(np.mean).reset_index()
df_inflation

,month,year,day,expected inflation
0,1,1978,1,5.2
1,1,1979,1,7.8
2,1,1980,1,10.4
3,1,1981,1,9.5
4,1,1982,1,5.1
...,...,...,...,...
533,12,2017,1,2.7
534,12,2018,1,2.7
535,12,2019,1,2.3
536,12,2020,1,2.5


In [6]:
# df_fares = pd.read_csv('uber fares/uber fares.csv')
df_fares = pd.read_csv('uber fares/uber fares.csv')#.sample(frac=0.5, replace=True)
df_fares = df_fares[['pickup_datetime', 'fare_amount', 'passenger_count']]
df_fares['pickup_datetime'] = pd.to_datetime(df_fares['pickup_datetime'])
df_fares['month'] = df_fares['pickup_datetime'].dt.month
df_fares['year'] = df_fares['pickup_datetime'].dt.year
df_fares['day'] = df_fares['pickup_datetime'].dt.day
df_fares = df_fares.groupby(['month', 'year', 'day']).agg(np.mean).reset_index()
df_fares

,month,year,day,fare_amount,passenger_count
0,1,2009,1,9.860317,1.793651
1,1,2009,2,12.325833,1.883333
2,1,2009,3,11.139286,1.750000
3,1,2009,4,9.777333,1.760000
4,1,2009,5,8.608594,1.703125
...,...,...,...,...,...
2367,12,2014,27,13.587941,1.544118
2368,12,2014,28,12.280980,1.784314
2369,12,2014,29,11.605294,2.176471
2370,12,2014,30,12.676324,1.852941


In [7]:
df_unemployment = pd.read_csv('unemployment_rate.csv').rename({'UNRATE': 'unemployment rate'}, axis=1)
df_unemployment['month'] = pd.to_datetime(df_unemployment['DATE']).dt.month
df_unemployment['year'] = pd.to_datetime(df_unemployment['DATE']).dt.year
df_unemployment['day'] = pd.to_datetime(df_unemployment['DATE']).dt.day
df_unemployment = df_unemployment.groupby(['month', 'year', 'day']).agg(np.mean).reset_index()
df_unemployment

,month,year,day,unemployment rate
0,1,1948,1,3.4
1,1,1949,1,4.3
2,1,1950,1,6.5
3,1,1951,1,3.7
4,1,1952,1,3.2
...,...,...,...,...
893,12,2017,1,4.1
894,12,2018,1,3.9
895,12,2019,1,3.6
896,12,2020,1,6.7


In [8]:
df_gas = pd.read_excel('sf gas data.xls', sheet_name='Data 1').drop([0,1]).reset_index(drop=True).rename({'Back to Contents': 'date'}, axis=1)
df_gas.columns.values[1] = 'gas price'
df_gas['date'] = pd.to_datetime(df_gas['date'])
df_gas['year'] = df_gas['date'].dt.year
df_gas['month'] = df_gas['date'].dt.month
df_gas['day'] = df_gas['date'].dt.day
df_gas['gas price'] = df_gas['gas price'].astype(float)
df_gas

,date,gas price,year,month,day
0,2000-06-05,1.759,2000,6,5
1,2000-06-12,1.766,2000,6,12
2,2000-06-19,1.769,2000,6,19
3,2000-06-26,1.785,2000,6,26
4,2000-07-03,1.836,2000,7,3
...,...,...,...,...,...
1168,2022-10-24,5.580,2022,10,24
1169,2022-10-31,5.353,2022,10,31
1170,2022-11-07,5.260,2022,11,7
1171,2022-11-14,5.255,2022,11,14


In [9]:
'''
Merging the dataframes to create the final df used in 2SLS
'''
merged_df1 = df_inflation.merge(right=df_unemployment, left_on=['month', 'year'], right_on=['month', 'year'])
merged_df1 = merged_df1.merge(right=df_gas, left_on=['month', 'year'], right_on=['month', 'year'])
merged_df2 = df_fares.merge(right=df_2015, left_on=['month', 'year', 'day'], right_on=['month', 'year', 'day'])
merged_df1 = merged_df1.groupby(['month', 'year']).agg(np.mean).reset_index().drop(['day'], axis=1)
merged_df = merged_df2.merge(merged_df1, left_on=['month', 'year'], right_on=['month', 'year']).drop(
    ['day_x', 'day_y'], axis=1)
merged_df
# merged_df2 = merged_df2.merge(right=df_fares, left_on=['month', 'year', 'day'], right_on=['month', 'year', 'day'])
# merged_df.merge(right=df_gas, left_on=['month', 'year'], right_on=['month', 'year'])

,month,year,day,fare_amount,passenger_count,number of rides,expected inflation,unemployment rate,gas price
0,1,2015,1,10.262576,1.651515,58045,2.5,5.7,2.62575
1,1,2015,2,13.945106,1.489362,40287,2.5,5.7,2.62575
2,1,2015,3,11.629342,1.802632,59844,2.5,5.7,2.62575
3,1,2015,4,13.294615,1.903846,41943,2.5,5.7,2.62575
4,1,2015,5,11.310392,1.784314,43678,2.5,5.7,2.62575
...,...,...,...,...,...,...,...,...,...
176,6,2015,26,13.359506,1.790123,99508,2.7,5.3,3.53500
177,6,2015,27,14.856533,1.773333,136193,2.7,5.3,3.53500
178,6,2015,28,13.932000,1.892308,93108,2.7,5.3,3.53500
179,6,2015,29,12.128889,1.571429,74430,2.7,5.3,3.53500


# Performing 2SLS

In [10]:
import statsmodels.api as sm

In [11]:
exog_2sls = sm.add_constant(merged_df[['unemployment rate', 'fare_amount', 'expected inflation']])
# w_2sls = merged_df['expected inflation']
z_2sls = merged_df['gas price']

C:\Users\carte\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [12]:
model_zhat = sm.OLS(z_2sls, exog_2sls)
results_zhat = model_zhat.fit(cov_type='HC1')
results_zhat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              gas price   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     96.19
Date:                Sun, 27 Nov 2022   Prob (F-statistic):           5.68e-37
Time:                        12:21:32   Log-Likelihood:                -50.471
No. Observations:                 181   AIC:                             108.9
Df Residuals:                     177   BIC:                             121.7
Df Model:                           3                                         
Covariance Type:                  HC1                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  5.6283      1.277      4.408      0.000       3.126       8.131
unemployment rate     -0.9382      0.178     -5.266      0.000      -1.287      -0.589
fare_amount            0.0688      0.015      4.580      0.000       0.039       0.098
expected inflation     0.6695      0.119      5.608      0.000       0.435       0.903
==============================================================================
Omnibus:                        5.529   Durbin-Watson:                   0.217
Prob(Omnibus):                  0.063   Jarque-Bera (JB):                3.993
Skew:                           0.229   Prob(JB):                        0.136
Kurtosis:                       2.434   Cond. No.                         800.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [13]:
merged_df['gas price hat'] = results_zhat.fittedvalues

In [14]:
y_2sls = merged_df['number of rides']
X_2sls = sm.add_constant(merged_df[['unemployment rate', 'fare_amount', 'gas price hat']])
model_yhat = sm.OLS(y_2sls, X_2sls)
results_yhat = model_yhat.fit(cov_type='HC1')
results_yhat.summary()

C:\Users\carte\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        number of rides   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     8.034
Date:                Sun, 27 Nov 2022   Prob (F-statistic):           4.76e-05
Time:                        12:21:32   Log-Likelihood:                -1993.8
No. Observations:                 181   AIC:                             3996.
Df Residuals:                     177   BIC:                             4008.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              2.238e+05    1.1e+05      2.033      0.042    8062.725     4.4e+05
unemployment rate -3.123e+04   1.58e+04     -1.978      0.048   -6.22e+04    -287.167
fare_amount         391.6351   1063.185      0.368      0.713   -1692.168    2475.438
gas price hat      6633.3149    1.1e+04      0.603      0.547   -1.49e+04    2.82e+04
==============================================================================
Omnibus:                        2.178   Durbin-Watson:                   0.753
Prob(Omnibus):                  0.337   Jarque-Bera (JB):                1.760
Skew:                           0.192   Prob(JB):                        0.415
Kurtosis:                       3.294   Cond. No.                     1.46e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
[2] The condition number is large, 1.46e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
'''
Naive OlS results
'''
y_test = merged_df['number of rides']
x_test = sm.add_constant(merged_df[['gas price', 'unemployment rate', 'fare_amount']])
model_test = sm.OLS(y_test, x_test)
results_test = model_test.fit(cov_type='HC1')
results_test.summary()

C:\Users\carte\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        number of rides   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     16.78
Date:                Sun, 27 Nov 2022   Prob (F-statistic):           1.22e-09
Time:                        12:29:31   Log-Likelihood:                -1983.8
No. Observations:                 181   AIC:                             3976.
Df Residuals:                     177   BIC:                             3988.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              1.546e+05   5.19e+04      2.982      0.003     5.3e+04    2.56e+05
gas price          1.441e+04   3049.646      4.725      0.000    8433.268    2.04e+04
unemployment rate -2.175e+04   8553.879     -2.543      0.011   -3.85e+04   -4988.910
fare_amount        -203.5822    773.898     -0.263      0.793   -1720.395    1313.230
==============================================================================
Omnibus:                        2.724   Durbin-Watson:                   0.778
Prob(Omnibus):                  0.256   Jarque-Bera (JB):                2.381
Skew:                           0.180   Prob(JB):                        0.304
Kurtosis:                       3.432   Cond. No.                         755.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

My 2SLS model is underspecified because the `gas price` variable has a positive coefficient, when it's likely that it should be negative. My model still performs better than the naive OLS regression coefficient (14,000 > 6,633).

There may be better instruments and more confounders that need to be identified to produce a better causal estimate of gas prices on the number of Uber rides.

# Predicting Uber Prices

In [16]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [17]:
train, test = train_test_split(merged_df, test_size=0.3)

In [18]:
X_train = train[['passenger_count', 'number of rides', 'expected inflation', 'unemployment rate', 'gas price']]
y_train = train['fare_amount']

X_test = test[['passenger_count', 'number of rides', 'expected inflation', 'unemployment rate', 'gas price']]
y_test = test['fare_amount']

In [19]:
# model_train = sm.OLS(y_train, X_train)
# results_train = model_train.fit(cov_type='HC1')
# results_train.summary()

In [20]:
# from statsmodels.tools.eval_measures import rmse

In [21]:
# rmse(y_train, results_train.fittedvalues)

In [22]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

In [23]:
lm.fit(X_train, y_train)

LinearRegression()

In [24]:
y_predicted = lm.predict(X_test)

In [25]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_predicted)))

RMSE: 1.7017087093185588


The RMSE of average daily pricing error is \$1.70 using linear regression and the following as predictors:

1. average `passenger_count` per day
2. `number of rides` per day
3. `expected inflation` per month
4. `unemployment rate` per month
5. `gas price` per week

Limitations: 

1. Data isn't as granular as would be best
2. Time frame limited to 2015